# **Sarvam Translate API Tutorial**

This notebook demonstrates how to use the **Sarvam translate API** to translate tests/paragraphs from one language to another. The API supports additional features such as transliteration(a type of conversion of a text from one script to another that involves swapping letters).



## **Step 1. Installation**

Before you begin, ensure you have the necessary Python libraries installed. Run the following commands to install the required packages:


In [1]:
!pip install requests


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


## **Step 2: Set Up API Key**


To use the Saaras API, you need an API subscription key. Follow these steps to set up your API key:

1. **Obtain your API key**: If you don’t have an API key, sign up on the [Sarvam AI Dashboard](https://dashboard.sarvam.ai/) to get one.
2. **Replace the placeholder key**: In the code below, replace "YOUR_SARVAM_AI_API_KEY" with your actual API key.

In [7]:
SARVAM_API_KEY = "key"

## **Step 3: Read the Document**

We have two sample documents under the `data` folder:  
- `sample1.txt` contains an essay on *The Impact of Artificial Intelligence on Society* in English.  
- `sample2.txt` contains the same essay in Hindi.


In [16]:
with open("sample1.txt", "r", encoding="utf-8") as file1:
    # Read first 5 lines
    lines = [next(file1) for _ in range(5)]  
    print("=== English Text (First Few Lines) ===")
    print("".join(lines))  # Print first few lines
    
    # Read the remaining content
    remaining_text = file1.read()

    # Combine all text
    full_text = "".join(lines) + remaining_text

    # Count total characters
    total_chars = len(full_text)
    print("\nTotal number of characters in file:", total_chars)


=== English Text (First Few Lines) ===
The Impact of Artificial Intelligence on Society

Artificial Intelligence (AI) has emerged as one of the most transformative technologies of the 21st century, revolutionizing various aspects of human life. From healthcare to finance, transportation to education, AI is reshaping industries, automating processes, and augmenting human capabilities. While AI presents numerous benefits, it also raises ethical, economic, and societal concerns that must be carefully navigated to ensure its responsible integration into society.

One of the most profound impacts of AI is in the field of healthcare. AI-powered diagnostic tools enable early disease detection, improving patient outcomes and reducing healthcare costs. Machine learning algorithms analyze medical data to identify patterns that humans may overlook, aiding in personalized treatment plans. Robotic surgeries and AI-driven drug discoveries have further enhanced the efficiency of medical procedures. H

## **Step 4: Sentence Splitting for Streaming**  

To enable real-time translation, we first split the text into individual sentences using **regex**, detecting boundaries like `.`, `!`, and `?`. This ensures **contextual clarity** and prevents broken translations.  

Each sentence is then sent to the API **one by one**, allowing translations to appear sequentially, creating a smooth streaming effect.


In [17]:
import re

def split_into_sentences(text):
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    return sentences

sentences_list = split_into_sentences(full_text)

# Print first 5 sentences (or all if fewer than 5)
print("\n=== First 5 Sentences ===")
for i, sentence in enumerate(sentences_list[:5], 1):  
    print(f"{i}. {sentence}")



=== First 5 Sentences ===
1. The Impact of Artificial Intelligence on Society

Artificial Intelligence (AI) has emerged as one of the most transformative technologies of the 21st century, revolutionizing various aspects of human life.
2. From healthcare to finance, transportation to education, AI is reshaping industries, automating processes, and augmenting human capabilities.
3. While AI presents numerous benefits, it also raises ethical, economic, and societal concerns that must be carefully navigated to ensure its responsible integration into society.
4. One of the most profound impacts of AI is in the field of healthcare.
5. AI-powered diagnostic tools enable early disease detection, improving patient outcomes and reducing healthcare costs.


## **Step 5: Calling the API Endpoint**  

In [18]:
import requests
import os
import re
import time

API_KEY = SARVAM_API_KEY 

if not API_KEY:
    raise ValueError("API key is missing! Set SARVAM_API_KEY in environment variables.")

url = "https://api.sarvam.ai/translate"
headers = {
    "api-subscription-key": API_KEY,
    "Content-Type": "application/json"
}

for i, sentence in enumerate(sentences_list, 1):

    payload = {
        "source_language_code": "en-IN",
        "target_language_code": "hi-IN",
        "speaker_gender": "Male",
        "mode": "classic-colloquial",
        "model": "mayura:v1",
        "enable_preprocessing": False,
        "input": sentence
    }

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        translation = response.json().get("translated_text", "Translation not available")
        print(f"{translation}")
    else:
        print(f"{i}. {sentence} ➝ [Translation Failed]")

    time.sleep(0.5)


Artificial Intelligence का समाज पर असर

Artificial Intelligence (AI) 21st century की सबसे बड़ी technologies में से एक है, जो इंसानी ज़िन्दगी के कई पहलुओं में बड़ा बदलाव लेकर आई है।
Healthcare से लेकर finance, transportation से लेकर education, AI industries को बदल रहा है, processes को automate कर रहा है, और इंसानी क्षमताओं को बढ़ा रहा है।
AI के बहुत फ़ायदे हैं, लेकिन इसके नैतिक, आर्थिक और सामाजिक मसले भी हैं, जिनसे हमें सावधानी से निपटना होगा ताकि समाज में इसका सही तरीक़े से इस्तेमाल हो सके।
AI का सबसे गहरा असर healthcare के क्षेत्र में है।
AI-powered diagnostic tools से बीमारी जल्दी पता चलती है, मरीज़ों के लिए अच्छा इलाज हो पाता है और healthcare costs कम होते हैं।
Machine learning algorithms medical data को analyze करते हैं, ताकि वो patterns ढूंढ सकें जो इंसानों की नज़र से छूट जाते हैं, और personalized treatment plans में मदद कर सकें।
Robotic surgeries और AI-driven drug discoveries ने medical procedures की efficiency और भी बढ़ा दी है।
लेकिन healthcare में AI के आने से data privacy, securi